In [1]:
import glob, os
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
# set up file directory and run_id
# when we have new runs, copy the 'building_data_2050.csv' output to this directory

run_dir = 'M:\\Data\\Urban\\BAUS\\PBA50\\Final_Blueprint\\run_analysis\\DR analysis'
os.chdir(run_dir)

# run_id dictionary
runid = {'run325': 'v2.2.1'}

In [3]:
# get parcel_id - geographies mapping
p_geo = pd.read_csv(r'M:\Data\Urban\BAUS\PBA50\Final_Blueprint\Large General Input Data\2020_09_21_parcels_geography.csv',
                    usecols = ['PARCEL_ID', 'juris',
                               #'fbpzoningm', 'fbp_gg_id', 'pda_id_pba50_fb', 'fbp_tra_id', 
                               'fbp_sesit_id'])
juris_county = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\petrale\zones\jurisdictions\juris_county_id.csv',
                           usecols = ['juris_name_full', 'county_name'])

In [4]:
juris = list(p_geo.juris.unique())
hra = ['HRA','non-HRA']
dis = ['DIS','non-DIS']

dr_juris_all = pd.Series(np.nan, index =juris, name='temp')
dr_hra_all = pd.Series(np.nan, index = hra, name='temp')
dr_dis_all = pd.Series(np.nan, index = dis, name='temp')
dr_juris_pub_all = pd.Series(np.nan, index =juris, name='temp')

writer = pd.ExcelWriter('deed_restricted_units_summary.xlsx', engine='xlsxwriter')
files = dict()

In [5]:
for file in list(glob.glob('*.csv')):
    print(runid[file.split('_')[0]])
    if 'building_data_2050.csv' in file:
        df = pd.read_csv(file,
                         usecols = ['parcel_id','source','residential_units','deed_restricted_units',
                                    'inclusionary_units','preserved_units', 'subsidized_units'])
        df = df.merge(p_geo, left_on='parcel_id', right_on='PARCEL_ID', how='left')
        
        # code HRA/non-HRA and DIS/non-DIS
        df['HRA'] = 'HRA'
        df.loc[(df.fbp_sesit_id == 'DIS') | (df.fbp_sesit_id.isnull()), 'HRA'] = 'non-HRA'
        df['DIS'] = 'DIS'
        df.loc[(df.fbp_sesit_id == 'HRA') | (df.fbp_sesit_id.isnull()), 'DIS'] = 'non-DIS'
        
        # visualize DR counts by jurisdiction
        dr_juris = df.groupby(['juris'])['deed_restricted_units'].sum()
        dr_juris.rename('dr_units_'+file.split('_')[0]+'_'+runid[file.split('_')[0]], inplace=True)
        #display(dr_juris)
        dr_juris_all = pd.concat([dr_juris_all, dr_juris], axis=1)
        
        dr_hra = df.groupby(['HRA'])['deed_restricted_units'].sum()
        dr_hra.rename('dr_units_'+file.split('_')[0]+'_'+runid[file.split('_')[0]], inplace=True)
        #display(dr_hra)
        dr_hra_all = pd.concat([dr_hra_all, dr_hra], axis=1)
        
        dr_dis = df.groupby(['DIS'])['deed_restricted_units'].sum()
        dr_dis.rename('dr_units_'+file.split('_')[0]+'_'+runid[file.split('_')[0]], inplace=True)
        #display(dr_dis)
        dr_dis_all = pd.concat([dr_dis_all, dr_dis], axis=1)
        
        dr_juris_pub = df.loc[df.source == 'pub'].groupby(['juris'])['deed_restricted_units'].sum()
        dr_juris_pub.rename('dr_units_publicLand_'+file.split('_')[0]+'_'+runid[file.split('_')[0]], inplace=True)
        #display(dr_juris_pub)
        dr_juris_pub_all = pd.concat([dr_juris_pub_all, dr_juris_pub], axis=1)

#         fig = go.Figure(go.Bar(
#             x = dr_juris['deed_restricted_units'],
#             y = ['giraffes', 'orangutans', 'monkeys'],
#             orientation='h'))

#         fig.show()

v2.2.1


In [6]:
dr_juris_all.reset_index(inplace=True)
dr_juris_all.drop(columns=['temp'], inplace=True)
dr_juris_all = dr_juris_all.merge(juris_county, left_on='index', right_on='juris_name_full', how='left')
dr_juris_all.drop(columns = ['juris_name_full'], inplace=True)
dr_juris_all.sort_values(by=['county_name', 'index'], inplace=True)
files['dr_juris'] = dr_juris_all

dr_hra_all.reset_index(inplace=True)
dr_hra_all.drop(columns=['temp'], inplace=True)
#display(dr_hra_all)
files['dr_hra'] = dr_hra_all

dr_dis_all.reset_index(inplace=True)
dr_dis_all.drop(columns=['temp'], inplace=True)
#display(dr_dis_all)
files['dr_dis'] = dr_dis_all

dr_juris_pub_all.reset_index(inplace=True)
dr_juris_pub_all.drop(columns=['temp'], inplace=True)
dr_juris_pub_all.fillna(0, inplace=True)
dr_juris_pub_all = dr_juris_pub_all.merge(juris_county, left_on='index', right_on='juris_name_full', how='left')
dr_juris_pub_all.drop(columns = ['juris_name_full'], inplace=True)
dr_juris_pub_all.sort_values(by=['county_name', 'index'], inplace=True)
#display(dr_juris_pub_all)
files['dr_juris_pub'] = dr_juris_pub_all

for name in files.keys():
    files[name].to_excel(writer, sheet_name=name, index=False)

writer.save()